## 1. Order images

After manually check the area of interest in planet explorer, a list containing all the item ids was created and stored in training_afk/0_aoi/1802_ids.txt file. 

By using a grid of 0.1x0.1 degrees and compared with the image ids, an order will be created with a clip tool and will be served in GEE.

In [ ]:
import backoff
import os
from pathlib import Path
import requests
from tqdm.auto import tqdm
from planet import api as planet_api
from time import sleep

from shapely.geometry import shape
from shapely_geojson import dumps
import geopandas as gpd
import pandas as pd
import json

In [ ]:
# Set session auth
PLANET_API_KEY = os.getenv("PLANET_API_KEY")
session = requests.Session()
session.auth = requests.auth.HTTPBasicAuth(PLANET_API_KEY, '')
session.headers.update({'Content-Type': 'application/json; charset=utf-8', 'Accept': 'application/json'})

# Set connection urls
orders_url = 'https://api.planet.com/compute/ops/orders/v2'

In [ ]:
ids_path = Path("training_afk/0_aoi/1802_ids.txt")
with open(ids_path) as f:
    item_ids = [l.strip() for l in f.read().split(",")]
    # item_ids = [line.replace("\n","") for line in f]

In [ ]:
fishnet = gpd.GeoDataFrame.from_file("training_afk/0_aoi/shp/aoi_fishnet.shp")
fishnet;

In [ ]:
# This dataframe was created by using some geoprocessings in QGIS
# To be sure the entire area was fully covered by images.
all_images_path = "training_afk/0_aoi/shp/all_foot_images.shp"
all_images_gdf = gpd.GeoDataFrame.from_file(all_images_path)
images_ids = all_images_gdf.id.unique()

In [ ]:
def extract_geometry(item_id):
    """Creates a geodataframe containing image id and its geometries (footprint)"""
    
    url=f"https://api.planet.com/data/v1/item-types/PSScene/items/{item_id}"
    response = requests.get(url, auth=session.auth)
    json = response.json()
    
    return {
        "id":json["id"], 
        "date":json["properties"]["acquired"],
        "geometry": shape(json["geometry"])
    }

image_geometries = [extract_geometry(item_id) for item_id in images_ids]

In [ ]:
image_gdf = gpd.GeoDataFrame(image_geometries)

In [ ]:
image_gdf["date"]=pd.to_datetime(image_gdf["date"]) 

In [ ]:
main_date = image_gdf.date.dt.strftime('%d-%m-%y').unique()[0]
image_gdf["date"]=image_gdf.date.dt.strftime('%d-%m-%y')
# image_gdf.to_file(f"0_west_afk/shp/images_geometry_{main_date}.shp", driver="ESRI Shapefile")
main_date

### 1.1 Get all images associated to each grid 

In [ ]:
fishnet;
all_images_gdf;

In [ ]:
"""Creates a dictionary with grid_id associated with planet id images"""

images_by_grid = {}

for idx, grid in fishnet.iterrows():
    
    images_by_grid[idx] = []
    
    for _, img in all_images_gdf.iterrows():

        if grid.geometry.intersects(img.geometry):

            images_by_grid[idx].append(img.id)

In [ ]:
@backoff.on_exception(
    backoff.expo,
    (planet_api.exceptions.OverQuota, planet_api.exceptions.TooManyRequests),
    max_time=360
)
def post_order(grid_id):
    

    order_request = {
        "name": str(grid_id),
        "order_type": "partial",
        "products": [
            {
                "item_type": "PSScene",
                "item_ids": images_by_grid[grid_id],
                "product_bundle": "analytic_8b_sr_udm2"
            }
        ],
        "tools": [
            {"clip": {"aoi": json.loads(dumps(fishnet.iloc[grid_id].geometry))}},
        ],
        "delivery": {
            "google_earth_engine": {
                "project": "planet-afk",
                "collection": "afk_treecount"
            }
        },
        "notifications": {"email": True},
    }
    return order_request
    # Send order to server
    return requests.post(
        orders_url, 
        data=json.dumps(order_request), 
        auth=session.auth, 
        headers=session.headers
    )
    

In [ ]:
responses = {}
for grid_id in tqdm(images_by_grid):
    if grid_id<=10: continue
    responses[grid_id] = post_order(grid_id)
    print(grid_id)
    sleep(1)

## 2. Create a random sample

In [ ]:
planet_collection = ee.ImageCollection("projects/planet-afk/assets/afk_treecount")
ee_fishnet = ee.FeatureCollection("projects/planet-afk/assets/aoi_fishnet")

In [ ]:
# Based on https://medium.com/google-earth/random-samples-with-buffering-6c8737384f8c

cellSize = 5000
seed = 1
crs = planet_collection.first().projection().getInfo()["crs"]

# Generate a random image of integers projected crs
proj = ee.Projection(crs).atScale(cellSize)
cells = ee.Image.random(seed).multiply(1000000).int().clip(fishnet).reproject(proj)

# Generate another random image and select the maximum random value 
# in each grid cell as the sample point.

random = ee.Image.random(seed).multiply(1000000).int()
maximum = cells.addBands(random).reduceConnectedComponents(ee.Reducer.max())
  
# Find all the points that are local maximums.
points = random.eq(maximum).selfMask().clip(fishnet).reproject(proj.scale(1/8, 1/8))
    
samples = points.reduceToVectors(**{
  "reducer": ee.Reducer.countEvery(), 
  "geometry": fishnet,
  "crs": proj.scale(1/8, 1/8), 
  "geometryType": "centroid", 
  "maxPixels": 1e9
})

# Add a buffer around each point that is the requested spacing size for visualization.
buffer = samples.map(lambda x: f.buffer(ee.Number(300)).bounds())


# Random sample
random_sample = ee.FeatureCollection(
    buffer.toList(buffer.size()).shuffle(6).slice(0,100)
)
  
task = ee.batch.export.table.toAsset(random_sample, "afk_random_areas", "afk_random_areas")
task.start()

## 3. Export images to drive

In [ ]:
import ee
ee.Initialize()

In [ ]:
planet_collection = ee.ImageCollection("projects/planet-afk/assets/afk_treecount")
ee_fishnet = ee.FeatureCollection("projects/planet-afk/assets/afk_random_areas")

In [ ]:
mosaic = planet_collection.mosaic()
band_names = mosaic.bandNames().slice(0,8)
composite = planet_collection.select(band_names).mean()

In [ ]:
n = ee_fishnet.size().getInfo()
n

In [ ]:
tasks = []
for patch_id in range(n):
    
    region = ee_fishnet.filter(ee.Filter.eq("id", patch_id))
    
    task = ee.batch.Export.image.toDrive(**{
        "image":composite, 
        "description":f"afk_mean_composite_area{patch_id}", 
        "folder":"planet_afk/random_areas",
        "scale": 3,
        "crs": "EPSG:4326",
        "region": region.geometry(),
        "maxPixels":1e13
    })
    task.start()
    tasks.append(task)

In [ ]:
# [tasks[i].start() for i, task in enumerate(tasks) ]

In [ ]:
import rasterio as rio
from pathlib import Path
import geopandas as gpd
from shapely.geometry import box
import rasterio.mask

In [ ]:
image = Path.home()/"1_modules/3_WADL/notebooks/training_afk/0_raw/annotation/aduna_crowns_rfclass_0_1.tif"
fishnet = gpd.GeoDataFrame.from_file("training_afk/0_aoi/shp/aoi_fishnet.shp")

In [ ]:
id_ = 5
aoi = fishnet[fishnet.id==id_]
bbox = box(*aoi.iloc[[0]].total_bounds)

In [ ]:
rio_image = rio.open(image)
profile = rio_image.profile

In [ ]:
sm = rio.mask.mask(rio_image, [bbox], all_touched=True, crop=True)

In [ ]:
profile["height"] = sm[0].shape[1]
profile["width"] = sm[0].shape[2]
profile["transform"] = sm[1]
profile["count"] = 1

In [ ]:
with rio.open(image.with_name(f"patch_{id_}").with_suffix(".tif"), "w", **profile) as dst:
    dst.write(sm[0][0], 1)